In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append('..')
sys.path.append(os.path.abspath('../trend_detection/'))
    
from datetime import datetime
import pytz
import json
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from datetime import timedelta
    
import map_utils
    
from gen_lib import data_saver, generation_utils
from gen_lib.config import *
from gen_lib.locations import LocationData
from gen_lib.grid_generator import generate_location_heatmap, VariationConfig, GridGeneratorEnhanced
from gen_lib.trends import TrendConfig, TrendGenerator, save_trend_data
from gen_lib.time_series import TimeSeriesHeatmapGenerator, TimeSeriesConfig

from stream_generation.azure_generator import AzureContentGenerator
from stream_generation import trends

from trend_detection_embeddings import TrendDetectorEmbeddings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
AZURE_OPENAI_ENDPOINT = "https://my-first-open-ai-service.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-08-01-preview"
AZURE_OPENAI_KEY = ""

generator = AzureContentGenerator(
    openai_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_key=AZURE_OPENAI_KEY,
    temperature=1.2,
)

In [51]:
generator.generate_message(trends.TRENDS[0])

{'trend_id': 1,
 'trend_name': 'AI Model Breakthrough',
 'topic': 'TECH',
 'text': '"Mind blown by the latest #AI Model Breakthrough. Human-level reasoning for complex problem-solving tasks! Big news for #TechInnovation & the #FutureTech landscape. Can\'t wait for live demos and ethical debates. #AGI"'}

In [ ]:
messages = []

for j, event in enumerate(trends.TRENDS):
    for i in range(100):
        msg = generator.generate_message(event)
        messages.append(msg)
        print(j, i)
        # print(msg['text'])
        # print('-'*80)

In [45]:
len(messages)

800

In [50]:
messages[0]

'"This AI breakthrough is mindblowing! The model can reason like a human on complex problem-solving tasks #AI #AGI #TechInnovation #FutureTech"'

In [48]:
df = pd.DataFrame(messages).rename(columns={'id': 'trend_id', 'name': 'trend_name'})
df.to_json('../stream_generation/data/messages_raw.json', orient='records')

In [44]:
td = TrendDetectorEmbeddings()

messages = list(df["text"])
random.shuffle(messages)

for i, message in enumerate(messages):
    for event in td.process_message(message, "LA", time.time()):
        print(event)

TrendEvent(trend_id='trend_1732691453_tech_model_futuretech_agi_techinnovation', trend_event='TREND_CREATED', trend_info='tech, model, futuretech, agi, techinnovation')
TrendEvent(trend_id='trend_1732691454_victory_championship_celebration_chicagosports_downtown', trend_event='TREND_CREATED', trend_info='victory, championship, celebration, chicagosports, downtown')
TrendEvent(trend_id='trend_1732691454_innovation_tech_techconference_summit_launches', trend_event='TREND_CREATED', trend_info='innovation, tech, techconference, summit, launches')
TrendEvent(trend_id='trend_1732691454_trading_financialcrisis_stockmarket_wallstreet_emergency', trend_event='TREND_CREATED', trend_info='trading, financialcrisis, stockmarket, wallstreet, emergency')
TrendEvent(trend_id='trend_1732691454_festival_beach_entertainment_miamimusic_music', trend_event='TREND_CREATED', trend_info='festival, beach, entertainment, miamimusic, music')
TrendEvent(trend_id='trend_1732691455_sports_championship_worldcupfinal

In [53]:
df2 = pd.read_json('../trend_detection/data/messages.json')
df2.head()

,trend_id,trend_name,topic,text
0,1,AI Model Breakthrough,TECH,"""Mind officially blown! Witnessed an AI model ..."
1,1,AI Model Breakthrough,TECH,"""This AI breakthrough is simply mind-blowing! ..."
2,1,AI Model Breakthrough,TECH,"""This AI breakthrough is mind-blowing! Human-l..."
3,1,AI Model Breakthrough,TECH,"""Interesting how #AI technology is advancing w..."
4,1,AI Model Breakthrough,TECH,"""This AI model breakthrough is taking the worl..."


In [57]:
df2.to_json('../stream_generation/data/messages_raw.json', orient='records')